# 代码分析

## 类继承关系
```mermaid
classDiagram
    %% 元类层次结构
    class type {
        <<metaclass>>
        +__new__(mcs, name, bases, namespace)
        +__init__(cls, name, bases, namespace)  
        +__call__(cls, *args, **kwargs)
    }
    
    class MetaStatsBuilderMixin {
        <<metaclass>>
        +metrics(cls) Config
    }
    
    %% 主要类
    class StatsBuilderMixin {
        +_metrics : Config
        -__init__()
        +writeable_attrs : Set[str]
        +stats_defaults : dict
        +metrics : Config
        +stats() Optional[BaseFigure]
        +build_metrics_doc(source_cls) str
        +override_metrics_doc(__pdoc__, source_cls) None
    }
    
    %% 继承关系
    type <|-- MetaStatsBuilderMixin
    MetaStatsBuilderMixin <|-- StatsBuilderMixin : metaclass
```

## class MetaStatsBuilderMixin(type)
```python
class MetaStatsBuilderMixin(type):
    @property
    def metrics(cls) -> Config:
        return cls._metrics
```

## class StatsBuilderMixin(metaclass=MetaStatsBuilderMixin)
被某类所继承，该类必须同时还继承 `Wrapping` 并且定义 `_metrics`，例如：

### 使用示例

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# 模拟 vectorbt 的相关组件
from vectorbt.base.array_wrapper import ArrayWrapper, Wrapping
from vectorbt.generic.stats_builder import StatsBuilderMixin
from vectorbt.utils.config import Config


class PortfolioAnalyzer(Wrapping, StatsBuilderMixin):
    """
    投资组合分析器示例类
    
    演示如何使用 StatsBuilderMixin 来计算各种投资组合统计指标
    """
    
    # 定义支持的统计指标配置
    _metrics = Config({
        # 基础收益指标
        'total_return': {
            'title': '总收益率',
            'calc_func': 'calculate_total_return',
            'tags': ['returns', 'basic'],
            'apply_to_timedelta': False
        },
        
        # 年化收益率
        'annualized_return': {
            'title': '年化收益率',
            'calc_func': 'calculate_annualized_return',
            'tags': ['returns', 'basic'],
            'apply_to_timedelta': False
        },
        
        # 波动率指标
        'volatility': {
            'title': '波动率',
            'calc_func': 'calculate_volatility',
            'tags': ['risk', 'volatility'],
            'apply_to_timedelta': False
        },
        
        # 年化波动率
        'annualized_volatility': {
            'title': '年化波动率',
            'calc_func': 'calculate_annualized_volatility',
            'tags': ['risk', 'volatility'],
            'apply_to_timedelta': False
        },
        
        # 夏普比率
        'sharpe_ratio': {
            'title': '夏普比率',
            'calc_func': 'calculate_sharpe_ratio',
            'tags': ['risk', 'ratio'],
            'apply_to_timedelta': False,
            'risk_free_rate': 0.02  # 默认无风险利率 2%
        },
        
        # 最大回撤
        'max_drawdown': {
            'title': '最大回撤',
            'calc_func': 'calculate_max_drawdown',
            'tags': ['risk', 'drawdown'],
            'apply_to_timedelta': False
        },
        
        # 卡尔马比率
        'calmar_ratio': {
            'title': '卡尔马比率',
            'calc_func': 'calculate_calmar_ratio',
            'tags': ['risk', 'ratio'],
            'apply_to_timedelta': False
        },
        
        # VaR (风险价值)
        'var_95': {
            'title': 'VaR (95%)',
            'calc_func': 'calculate_var',
            'tags': ['risk', 'var'],
            'apply_to_timedelta': False,
            'confidence_level': 0.95
        },
        
        # 胜率
        'win_rate': {
            'title': '胜率',
            'calc_func': 'calculate_win_rate',
            'tags': ['performance', 'ratio'],
            'apply_to_timedelta': False
        },
        
        # 盈亏比
        'profit_loss_ratio': {
            'title': '盈亏比',
            'calc_func': 'calculate_profit_loss_ratio',
            'tags': ['performance', 'ratio'],
            'apply_to_timedelta': False
        },
        
        # 索提诺比率
        'sortino_ratio': {
            'title': '索提诺比率',
            'calc_func': 'calculate_sortino_ratio',
            'tags': ['risk', 'ratio'],
            'apply_to_timedelta': False,
            'risk_free_rate': 0.02
        }
    })
    
    def __init__(self, returns_data: pd.Series, initial_value: float = 100000):
        """
        初始化投资组合分析器
        
        Args:
            returns_data: 收益率序列 (pd.Series)
            initial_value: 初始投资金额，默认 100,000
        """
        # 验证输入数据
        if not isinstance(returns_data, pd.Series):
            raise ValueError("returns_data 必须是 pd.Series 类型")
        
        # 创建 ArrayWrapper
        wrapper = ArrayWrapper.from_obj(returns_data)
        
        # 调用父类初始化
        Wrapping.__init__(self, wrapper)
        StatsBuilderMixin.__init__(self)
        
        # 存储数据
        self._returns = returns_data
        self._initial_value = initial_value
        
        # 计算累积净值
        self._cumulative_returns = (1 + self._returns).cumprod()
        self._portfolio_values = self._cumulative_returns * self._initial_value
        
        # 计算每日收益
        self._daily_returns = self._returns
        
        # 设置交易日年化因子
        self._annual_factor = 252  # 假设一年252个交易日
        
    @property
    def returns(self) -> pd.Series:
        """获取收益率序列"""
        return self._returns
    
    @property
    def portfolio_values(self) -> pd.Series:
        """获取投资组合价值序列"""
        return self._portfolio_values
    
    @property
    def cumulative_returns(self) -> pd.Series:
        """获取累积收益率序列"""
        return self._cumulative_returns
    
    # 统计指标计算方法
    
    def calculate_total_return(self) -> float:
        """计算总收益率"""
        return (self._portfolio_values.iloc[-1] / self._initial_value) - 1
    
    def calculate_annualized_return(self) -> float:
        """计算年化收益率"""
        total_return = self.calculate_total_return()
        periods = len(self._returns)
        years = periods / self._annual_factor
        return (1 + total_return) ** (1 / years) - 1
    
    def calculate_volatility(self) -> float:
        """计算波动率 (日频)"""
        return self._daily_returns.std()
    
    def calculate_annualized_volatility(self) -> float:
        """计算年化波动率"""
        daily_vol = self.calculate_volatility()
        return daily_vol * np.sqrt(self._annual_factor)
    
    def calculate_sharpe_ratio(self, risk_free_rate: float = 0.02) -> float:
        """计算夏普比率"""
        annualized_return = self.calculate_annualized_return()
        annualized_vol = self.calculate_annualized_volatility()
        
        if annualized_vol == 0:
            return 0.0
        
        return (annualized_return - risk_free_rate) / annualized_vol
    
    def calculate_max_drawdown(self) -> float:
        """计算最大回撤"""
        # 计算历史最高点
        rolling_max = self._portfolio_values.expanding().max()
        # 计算回撤
        drawdowns = (self._portfolio_values - rolling_max) / rolling_max
        # 返回最大回撤 (负值)
        return drawdowns.min()
    
    def calculate_calmar_ratio(self) -> float:
        """计算卡尔马比率 (年化收益率 / 最大回撤的绝对值)"""
        annualized_return = self.calculate_annualized_return()
        max_dd = abs(self.calculate_max_drawdown())
        
        if max_dd == 0:
            return float('inf') if annualized_return > 0 else 0.0
        
        return annualized_return / max_dd
    
    def calculate_var(self, confidence_level: float = 0.95) -> float:
        """计算风险价值 (VaR)"""
        return np.percentile(self._daily_returns, (1 - confidence_level) * 100)
    
    def calculate_win_rate(self) -> float:
        """计算胜率 (正收益日数占比)"""
        positive_days = (self._daily_returns > 0).sum()
        total_days = len(self._daily_returns)
        return positive_days / total_days
    
    def calculate_profit_loss_ratio(self) -> float:
        """计算盈亏比 (平均盈利 / 平均亏损的绝对值)"""
        profits = self._daily_returns[self._daily_returns > 0]
        losses = self._daily_returns[self._daily_returns < 0]
        
        if len(losses) == 0:
            return float('inf') if len(profits) > 0 else 0.0
        
        avg_profit = profits.mean()
        avg_loss = abs(losses.mean())
        
        return avg_profit / avg_loss if avg_loss != 0 else float('inf')
    
    def calculate_sortino_ratio(self, risk_free_rate: float = 0.02) -> float:
        """计算索提诺比率 (只考虑下行风险的夏普比率)"""
        annualized_return = self.calculate_annualized_return()
        
        # 计算下行偏差
        negative_returns = self._daily_returns[self._daily_returns < 0]
        if len(negative_returns) == 0:
            return float('inf') if annualized_return > risk_free_rate else 0.0
        
        downside_deviation = negative_returns.std() * np.sqrt(self._annual_factor)
        
        if downside_deviation == 0:
            return float('inf') if annualized_return > risk_free_rate else 0.0
        
        return (annualized_return - risk_free_rate) / downside_deviation


def create_sample_portfolio_data(days: int = 252, annual_return: float = 0.08, 
                               annual_volatility: float = 0.15, seed: int = 42) -> pd.Series:
    """
    创建模拟的投资组合收益率数据
    
    Args:
        days: 数据天数，默认252 (一年)
        annual_return: 年化收益率，默认8%
        annual_volatility: 年化波动率，默认15%
        seed: 随机种子
    
    Returns:
        pd.Series: 日收益率序列
    """
    np.random.seed(seed)
    
    # 转换为日收益率参数
    daily_return = annual_return / 252
    daily_volatility = annual_volatility / np.sqrt(252)
    
    # 生成随机收益率
    returns = np.random.normal(daily_return, daily_volatility, days)
    
    # 创建日期索引
    start_date = datetime(2023, 1, 1)
    dates = [start_date + timedelta(days=i) for i in range(days)]
    
    return pd.Series(returns, index=dates, name='daily_returns')


def main():
    """主函数：演示 StatsBuilderMixin 的完整使用"""
    
    print("=== StatsBuilderMixin 使用示例 ===\n")
    
    # 1. 创建模拟数据
    print("1. 创建模拟投资组合数据...")
    portfolio_returns = create_sample_portfolio_data(
        days=252,  # 一年的数据
        annual_return=0.12,  # 12% 年化收益
        annual_volatility=0.18,  # 18% 年化波动率
        seed=42
    )
    print(f"   数据长度: {len(portfolio_returns)} 天")
    print(f"   日期范围: {portfolio_returns.index[0].date()} 到 {portfolio_returns.index[-1].date()}")
    print(f"   平均日收益率: {portfolio_returns.mean():.4f}")
    print(f"   日收益率标准差: {portfolio_returns.std():.4f}")
    
    # 2. 创建分析器
    print("\n2. 创建投资组合分析器...")
    analyzer = PortfolioAnalyzer(portfolio_returns, initial_value=100000)
    print(f"   分析器类型: {type(analyzer)}")
    print(f"   可用指标: {list(analyzer.metrics.keys())}")
    print(f"   初始投资金额: ${analyzer._initial_value:,.2f}")
    print(f"   最终投资组合价值: ${analyzer.portfolio_values.iloc[-1]:,.2f}")
    
    # 3. 计算所有统计指标
    print("\n3. 计算所有统计指标...")
    all_stats = analyzer.stats()
    print("   所有指标:")
    for metric, value in all_stats.items():
        if isinstance(value, float):
            if abs(value) < 0.01:
                print(f"     {metric}: {value:.6f}")
            else:
                print(f"     {metric}: {value:.4f}")
        else:
            print(f"     {metric}: {value}")
    
    # 4. 按标签分组计算
    print("\n4. 按标签分组计算指标...")
    
    # 只计算收益相关指标
    returns_stats = analyzer.stats(tags=['returns'])
    print("   收益指标:")
    for metric, value in returns_stats.items():
        print(f"     {metric}: {value:.4f}")
    
    # 只计算风险相关指标
    risk_stats = analyzer.stats(tags=['risk'])
    print("   风险指标:")
    for metric, value in risk_stats.items():
        if isinstance(value, float):
            print(f"     {metric}: {value:.4f}")
        else:
            print(f"     {metric}: {value}")
    
    # 只计算比率指标
    ratio_stats = analyzer.stats(tags=['ratio'])
    print("   比率指标:")
    for metric, value in ratio_stats.items():
        print(f"     {metric}: {value:.4f}")
    
    # 5. 计算特定指标
    print("\n5. 计算特定指标...")
    specific_metrics = ['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate']
    specific_stats = analyzer.stats(metrics=specific_metrics)
    print("   指定指标:")
    for metric, value in specific_stats.items():
        print(f"     {metric}: {value:.4f}")
    
    # 6. 使用自定义参数
    print("\n6. 使用自定义参数...")
    custom_stats = analyzer.stats(
        metrics=['sharpe_ratio', 'sortino_ratio'],
        metric_settings={
            'sharpe_ratio': {'risk_free_rate': 0.03},  # 3% 无风险利率
            'sortino_ratio': {'risk_free_rate': 0.03}
        }
    )
    print("   自定义参数指标 (3% 无风险利率):")
    for metric, value in custom_stats.items():
        print(f"     {metric}: {value:.4f}")
    
    # 7. 使用模板参数
    print("\n7. 使用直接数值参数...")
    try:
        # 方法1：直接使用数值（最可靠）
        direct_stats = analyzer.stats(
            metrics=['sharpe_ratio', 'sortino_ratio'],
            metric_settings={
                'sharpe_ratio': {'risk_free_rate': 0.025},
                'sortino_ratio': {'risk_free_rate': 0.025}
            }
        )
        print("   直接参数指标 (2.5% 无风险利率):")
        for metric, value in direct_stats.items():
            print(f"     {metric}: {value:.4f}")
            
    except Exception as e:
        print(f"   错误: {e}")
        
        # 备用方法：分别计算
        print("   使用备用方法分别计算...")
        sharpe_025 = analyzer.calculate_sharpe_ratio(risk_free_rate=0.025)
        sortino_025 = analyzer.calculate_sortino_ratio(risk_free_rate=0.025)
        print(f"     sharpe_ratio: {sharpe_025:.4f}")
        print(f"     sortino_ratio: {sortino_025:.4f}")
    
    # 8. 性能评估总结
    print("\n8. 投资组合性能评估总结...")
    total_return = all_stats['总收益率']
    annual_return = all_stats['年化收益率']
    annual_vol = all_stats['年化波动率']
    sharpe = all_stats['夏普比率']
    max_dd = all_stats['最大回撤']
    win_rate = all_stats['胜率']
    
    print(f"   📈 总收益率: {total_return:.2%}")
    print(f"   📊 年化收益率: {annual_return:.2%}")
    print(f"   📉 年化波动率: {annual_vol:.2%}")
    print(f"   ⚖️  夏普比率: {sharpe:.3f}")
    print(f"   📉 最大回撤: {max_dd:.2%}")
    print(f"   🎯 胜率: {win_rate:.2%}")
    
    # 性能评级
    performance_grade = "优秀" if sharpe > 1.5 else "良好" if sharpe > 1.0 else "一般" if sharpe > 0.5 else "较差"
    print(f"   🏆 综合评级: {performance_grade}")
    
    return analyzer, all_stats


if __name__ == "__main__":
    analyzer, stats = main()
    
    # 可选：显示更多详细信息
    print(f"\n=== 详细分析 ===")
    print(f"投资期间: {analyzer.returns.index[0].date()} - {analyzer.returns.index[-1].date()}")
    print(f"交易天数: {len(analyzer.returns)} 天")
    print(f"正收益天数: {(analyzer.returns > 0).sum()} 天")
    print(f"负收益天数: {(analyzer.returns < 0).sum()} 天")
    print(f"最大单日收益: {analyzer.returns.max():.4f}")
    print(f"最大单日损失: {analyzer.returns.min():.4f}")

=== StatsBuilderMixin 使用示例 ===

1. 创建模拟投资组合数据...
   数据长度: 252 天
   日期范围: 2023-01-01 到 2023-09-09
   平均日收益率: 0.0004
   日收益率标准差: 0.0110

2. 创建投资组合分析器...
   分析器类型: <class '__main__.PortfolioAnalyzer'>
   可用指标: ['total_return', 'annualized_return', 'volatility', 'annualized_volatility', 'sharpe_ratio', 'max_drawdown', 'calmar_ratio', 'var_95', 'win_rate', 'profit_loss_ratio', 'sortino_ratio']
   初始投资金额: $100,000.00
   最终投资组合价值: $109,874.26

3. 计算所有统计指标...
   所有指标:
     总收益率: 0.0987
     年化收益率: 0.0987
     波动率: 0.0110
     年化波动率: 0.1741
     夏普比率: 0.4523
     最大回撤: -0.1555
     卡尔马比率: 0.6351
     VaR (95%): -0.0165
     胜率: 0.5317
     盈亏比: 0.9731
     索提诺比率: 0.8259

4. 按标签分组计算指标...
   收益指标:
     总收益率: 0.0987
     年化收益率: 0.0987
   风险指标:
     波动率: 0.0110
     年化波动率: 0.1741
     夏普比率: 0.4523
     最大回撤: -0.1555
     卡尔马比率: 0.6351
     VaR (95%): -0.0165
     索提诺比率: 0.8259
   比率指标:
     夏普比率: 0.4523
     卡尔马比率: 0.6351
     胜率: 0.5317
     盈亏比: 0.9731
     索提诺比率: 0.8259

5. 计算特定指标...
   指定指标:
  

### `__init__`
```python
def __init__(self) -> None:
    self._metrics = self.__class__._metrics.copy()
```

### writeable_attrs
```python
@property
def writeable_attrs(self) -> tp.Set[str]:
    return {'_metrics'}
```

### stats_defaults
```python
@property
def stats_defaults(self) -> tp.Kwargs:
    from vectorbt._settings import settings
    stats_builder_cfg = settings['stats_builder']

    return merge_dicts(
        stats_builder_cfg,
        dict(settings=dict(freq=self.wrapper.freq))
    )
```

### metrics
```python
@property
def metrics(self) -> Config:
    return self._metrics
```

### build_metrics_doc
```python
@classmethod
def build_metrics_doc(cls, source_cls: tp.Optional[type] = None) -> str:
    if source_cls is None:
        source_cls = StatsBuilderMixin  # 默认使用基类作为文档模板源
    
    # 使用字符串模板进行文档生成
    return string.Template(
        inspect.cleandoc(get_dict_attr(source_cls, 'metrics').__doc__)  # 获取模板文档字符串
    ).substitute(
        {
            'metrics': cls.metrics.to_doc(),  # 将指标配置转换为文档格式
            'cls_name': cls.__name__  # 插入当前类名
        }
    )
```

### override_metrics_doc
```python
@classmethod
def override_metrics_doc(cls, __pdoc__: dict, source_cls: tp.Optional[type] = None) -> None:
    __pdoc__[cls.__name__ + '.metrics'] = cls.build_metrics_doc(source_cls=source_cls)
```